In [1]:
# Minicons Installation
# Introduction can be found https://kanishka.xyz/post/minicons-running-large-scale-behavioral-analyses-on-transformer-lms/
# Tutorial and code can be found https://github.com/kanishkamisra/minicons/blob/master/examples/surprisals.md
#!pip install minicons

from minicons import scorer
import pandas as pd
import numpy as np
import json
import csv
import re
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import torch
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

/tmp/ipykernel_1820720/658913012.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
model_path = "gpt2-small/checkpoint-trainedtokenizer_10M_whitespace"
#model_path = "gpt2-small/checkpoint-trainedtokenizer_100M_whitespace"
#model_path = "gpt3-small/checkpoint-trainedtokenizer_10M_whitespace"
#model_path = "gpt3-small/checkpoint-trainedtokenizer_100M_whitespace"

model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2TokenizerFast.from_pretrained(model_path)

# wrap with minicons scorer
lm_scorer = scorer.IncrementalLMScorer(model_path, device = "cpu")

In [3]:
def calculate_surprisal(sentence):
    '''
    Takes in a sentence, and outputs surprisal values for each word.
    '''
    
    input_sentence = sentence
    # token_score() function of Minicons takes in several parameters
    # if surprisal = True, the output value is surprisal instead of log likelihood
    # if base_two = True, the log likelihood will be in base 2
    # see Minicons documentations for details
    # score tokens
    token_surprisals = lm_scorer.token_score(input_sentence, surprisal = True, base_two = True)[0]
    #print(token_surprisals)

    '''
    # filter out special tokens (like <pad>)
    special_tokens = set(tokenizer.all_special_tokens + ['<pad>'])
    filtered = [
        (token, score)
        for (token, score) in token_surprisals
        if token not in special_tokens
    ]
    '''

   # expand tokens that contain multiple words
    expanded = []
    for token, score in token_surprisals:
    #for token, score in filtered:
        token = token.strip('Ġ') # remove space marker
        if token.count('Ġ') > 0:
            # multiple words inside
            words = token.split('Ġ')
            words = [word for word in words if word]  # remove empty strings
            for i, word in enumerate(words):
                expanded_token = word
                expanded.append((expanded_token, score / len(words)))  # split surprisal equally
        else:
            expanded.append((token, score))    
    #print(expanded)

    # use regex to split into words and punctuation
    words = re.findall(r'\w+|[^\w\s]', sentence)
    results = []

    token_pointer = 0

    for word in words:
        accumulated = ''
        word_surprisal = 0.0

        while token_pointer < len(expanded):
            token, surprisal = expanded[token_pointer]
            accumulated += token
            word_surprisal += surprisal
            token_pointer += 1

            if accumulated == word:
                results.append((word, word_surprisal))
                break
        else:
            results.append((word, word_surprisal))

    return results


sentence = 'I know that your friend gave a baguette to Mary last weekend.'
calculate_surprisal(sentence)

[('I', 0.0),
 ('know', 5.239012718200684),
 ('that', 3.954333782196045),
 ('your', 10.374737739562988),
 ('friend', 6.853007793426514),
 ('gave', 12.485967636108398),
 ('a', 4.920528888702393),
 ('baguette', 40.79525184631348),
 ('to', 8.777816772460938),
 ('Mary', 12.327611923217773),
 ('last', 11.225496292114258),
 ('weekend', 8.15101146697998),
 ('.', 10.069524765014648)]

In [4]:
def calculate_sentence_surprisal(word_surprisals):
    '''Returns total surprisal and average surprisal per word.'''
    
    scores = [score for word, score in word_surprisals]
    total = sum(scores)
    avg = total / len(scores)
    return total, avg

"""
def sum_region_surprisal(row, region_list):
    '''Sums surprisals of a specified sentence region, extracting the relevant part from sentence_surprisals using index information.'''

    sentence_surprisals = row['surprisals']
    
    # retrieve all sentence column(from 'prefix' till 'end')
    all_columns = row.index.tolist()  # get the list of column names
    prefix_index = all_columns.index('prefix')
    eos_index = all_columns.index('end')
    
    # reconstruct the full sentence from 'prefix' to 'end' (exclude NaN values)
    full_sentence = ' '.join([str(row[col]) if pd.notna(row.get(col)) else '' for col in all_columns[prefix_index:eos_index + 1]])
    
    # use regex to split the full sentence into words & punctuation
    sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)
    
    # get the start and end indices for the region_list within the full sentence
    first_region_column = region_list[0]
    first_region_start_index = all_columns.index(first_region_column)
    
    last_region_column = region_list[-1]
    last_region_end_index = all_columns.index(last_region_column)
    
    # extract the relevant slice of sentence_surprisals that corresponds to the region_list
    # calculate the starting and ending index for the region_list part
    start_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:first_region_start_index]
        if pd.notna(row[col]) 
    ])
    end_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:last_region_end_index + 1]
        if pd.notna(row[col]) 
    ])

    # extract the slice of sentence_surprisals corresponding to the region_list part
    relevant_surprisals = sentence_surprisals[start_index:end_index]
    region_surprisal = sum([score for token, score in relevant_surprisals])
    
    return region_surprisal
"""

def sum_region_surprisal(row, region_list, priority_region = None, normalize = False):
    '''Sums surprisals of a specified sentence region with optional priority region handling.'''

    sentence_surprisals = row['surprisals']
    
    # retrieve all sentence column(from 'prefix' till 'end')
    all_columns = row.index.tolist()  # get the list of column names
    prefix_index = all_columns.index('prefix')
    eos_index = all_columns.index('end')
    
    # reconstruct the full sentence from 'prefix' to 'end' (exclude NaN values)
    full_sentence = ' '.join([str(row[col]) if pd.notna(row.get(col)) else '' for col in all_columns[prefix_index:eos_index + 1]])
    
    # use regex to split the full sentence into words & punctuation
    sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)
    
    if priority_region:
        # check if priority_region has non-zero surprisal
        if pd.notna(row[priority_region]):
            # extract text for the priority_region
            priority_region_index = all_columns.index(priority_region)
            
            start_index = sum([
                len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:priority_region_index]
                if pd.notna(row[col]) 
            ])
            end_index = sum([
                len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:priority_region_index + 1]
                if pd.notna(row[col]) 
            ])
            
            # calculate the surprisal for the priority region
            relevant_surprisals = sentence_surprisals[start_index:end_index]
            priority_surprisal = sum([score for token, score in relevant_surprisals])
    
            # return priority region's surprisal score if it is non-zero
            if priority_surprisal != 0:
                return priority_surprisal
    
    # if priority region is not provided or its surprisal is zero, calculate sum of region_list surprisals

    # get the start and end indices for the region_list within the full sentence
    first_region_column = region_list[0]
    first_region_start_index = all_columns.index(first_region_column)
    
    last_region_column = region_list[-1]
    last_region_end_index = all_columns.index(last_region_column)
    
    # extract the relevant slice of sentence_surprisals that corresponds to the region_list
    # calculate the starting and ending index for the region_list part
    start_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:first_region_start_index]
        if pd.notna(row[col]) 
    ])
    end_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:last_region_end_index + 1]
        if pd.notna(row[col]) 
    ])

    # extract the slice of sentence_surprisals corresponding to the region_list part
    relevant_surprisals = sentence_surprisals[start_index:end_index]
    region_surprisal = sum([score for token, score in relevant_surprisals])
    
    if normalize == False:
        return region_surprisal
    elif normalize == True:
        return region_surprisal / len(sentence_units)

In [5]:
def split_ends(ends):
    '''Splits off sentence-final punctuation.'''
    
    match = re.match(r'^(.*?)([.!?])$', ends.strip())
    if match:
        return pd.Series([match.group(1), match.group(2)])
    else:
        return pd.Series([ends, ''])  # no end punctuation
    
def encode_wh_licensor(df):
    '''
    Adds a numeric column to the DataFrame:
    - 1 if licensor is a wh-licensor
    - -1 otherwise (licensor is "that" or absent)
    '''
    df['wh_numeric'] = df['filler'].apply(lambda x: 1 if x > 0 else -1)
    return df

def merge_sentence(row, syntactic_parts):
    '''Merges sentence columns back into full sentence.'''
    
    parts = [row[part] for part in syntactic_parts]         
    # filter out NaN or empty parts
    non_empty_parts = [str(part).strip() for part in parts if pd.notna(part) and str(part).strip() != '']
    # join with spaces and add end punctuation
    sentence = ' '.join(non_empty_parts) + (row['end'] if pd.notna(row['end']) else '')
    return sentence

#### Statistical Analysis: Mixed-Effects Linear Regression Model

In [6]:
from IPython.display import display
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

def print_summary(result):
    '''
    Prints the descriptive header from a statsmodels MixedLMResults summary,
    and displays both the fixed-effects and random-effects components as pandas DataFrames.
    '''
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        
        summary_str = str(result.summary())
        lines = summary_str.split('\n')

        # PART 1: HEADER
        print("\n=== MODEL SUMMARY ===")
        for line in lines:
            if 'Coef.' in line:
                break  # stop when reaching the coefficient table
            pairs = re.findall(r'(\S[^:]*):\s+([^\s][^:]*?)(?=\s{2,}|$)', line)
            for key, value in pairs:
                print(f"{key.strip()}: {value.strip()}")

        # PART 2a: FIXED EFFECTS TABLE
        fixed_table = []
        in_fixed_table = False
        random_start_idx = None

        for i, line in enumerate(lines):
            if 'Coef.' in line:
                in_fixed_table = True
                continue
            if in_fixed_table:
                if not line.strip() or set(line.strip()) in [{'='}, {'-'}]:
                    continue  # skip empty/separator lines
                parts = line.strip().split()
                if len(parts) >= 7:
                    param = parts[0]
                    row = parts[1:7]
                    fixed_table.append([param] + row)
                else:
                    random_start_idx = i
                    break

        if fixed_table:
            df_fixed = pd.DataFrame(fixed_table, columns = [
                'Parameter', 'Coef.', 'Std.Err.', 'z', 'P>|z|', '[0.025', '0.975]'
            ])
            print("\n=== FIXED EFFECTS COEFFICIENTS ===")
            display(df_fixed)
        else:
            print("\nNo fixed-effects data found.")

        # PART 2b: RANDOM EFFECTS / VARIANCE COMPONENTS TABLE
        random_table = []
        if random_start_idx:
            for line in lines[random_start_idx:]:
                if not line.strip() or set(line.strip()) in [{'='}, {'-'}]:
                    continue
                parts = line.strip().split()
                # Look for last token ending in Var or Cov
                varcov_idx = None
                for j in reversed(range(len(parts))):
                    if parts[j].endswith('Var') or parts[j].endswith('Cov'):
                        varcov_idx = j
                        break
                if varcov_idx is not None:
                    param = ' '.join(parts[:varcov_idx + 1])
                    numeric_parts = parts[varcov_idx + 1:]
                    if len(numeric_parts) == 2:
                        val1, val2 = numeric_parts
                    elif len(numeric_parts) == 1:
                        val1, val2 = numeric_parts[0], ''
                    else:
                        val1, val2 = '', ''
                    random_table.append([param, val1, val2])

        if random_table:
            df_random = pd.DataFrame(random_table, columns = ['Parameter', 'Coef.', 'Std.Err.'])
            print("\n=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===")
            display(df_random)
        else:
            print("\nNo random-effects data found.")


In [7]:
# columns: item_id (indicates sentence set), wh_licensor (0/1), gap (0/1), island_type, surprisal

warnings.simplefilter("ignore", ConvergenceWarning)

def mixed_effects_linear_regression(df, iv_list, surprisal, label):
    '''
    Fits mixed-effects model and extracts wh-licensing interaction.
    '''
    '''    
    for field in iv_list:
        if field == 'wh_numeric' or 'gap_numeric' or 'gap_distance_length':
            pass
        else:
            df[field] = df[field].astype('category')
    '''

    interaction_terms = ' * '.join(iv_list)
    random_effects = ' + '.join(iv_list)

    model = smf.mixedlm(
        f"{surprisal} ~ {interaction_terms}",
        df,
        groups = df["item_id"],
        #re_formula = f"~{random_effects}"
        re_formula = "1"
        )

    result = model.fit()
    #interaction_coef = result.params.get('wh_licensor[T.1]:gap[T.1]', None)

    print(f"\n=== {label.upper()} ===")
    #print(result.summary())
    print_summary(result)

    return result

#interaction = mixed_effects_linear_regression(df, "construction_type") # label name to be changed according to construction type

#### Gap Distance - Categorical/Continuous

In [8]:
sentence_df = pd.read_csv('test_sentences/Gap Distance.csv')
sentence_df['gap_distance'] = sentence_df['gap_distance'].fillna('null')
#sentence_df['gap_distance'].dropna(inplace = True)
#sentence_df = sentence_df[sentence_df['gap_distance'].replace(['nan', 'NaN'], np.nan).notna()]

In [9]:
sentence_df[['temp_mod', 'end']] = sentence_df['temp_mod'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'licensor', 'subj', 'modifier', 'verb', 'object', 'prep', 'goal', 'temp_mod']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['gap_distance_length'] = sentence_df['modifier'].apply(lambda x: len(x) if pd.notna(x) else 0)
sentence_df['gap_distance'] = pd.Categorical(sentence_df['gap_distance'], categories = ['null', 'short', 'medium', 'long'], ordered = True)

Gap Position == Object

In [10]:
# modifier: short_mod, med_mod, long_mod

In [11]:
object_df = sentence_df.copy()
#object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['prep', 'goal']), axis = 1)
object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['prep', 'goal'], 'object'), axis = 1)

# filter for object gap
objectgap_df = object_df[object_df['gap_position'] == 'DO']

In [12]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(objectgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at object')


=== GAP DISTANCE: GAP POSITION AT OBJECT ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 37.0301
Min. group size: 16
Log-Likelihood: -1039.2963
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,32.150,1.334,24.102,0.000,29.536,34.764
1,gap_distance[T.short],-0.215,0.962,-0.224,0.823,-2.101,1.670
2,gap_distance[T.medium],-0.994,0.962,-1.033,0.302,-2.880,0.892
3,gap_distance[T.long],-1.327,0.962,-1.379,0.168,-3.213,0.559
4,wh_numeric,0.174,0.680,0.256,0.798,-1.159,1.508
5,wh_numeric:gap_distance[T.short],-0.105,0.962,-0.109,0.913,-1.991,1.781
6,wh_numeric:gap_distance[T.medium],-0.111,0.962,-0.116,0.908,-1.997,1.774
7,wh_numeric:gap_distance[T.long],-0.149,0.962,-0.154,0.877,-2.034,1.737
8,gap_numeric,-1.566,0.680,-2.302,0.021,-2.899,-0.232
9,gap_numeric:gap_distance[T.short],-0.147,0.962,-0.153,0.879,-2.033,1.739



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,26.328,1.577


In [13]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(objectgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at object')


=== GAP DISTANCE: GAP POSITION AT OBJECT ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 35.9921
Min. group size: 16
Log-Likelihood: -1062.3350
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,32.189,1.266,25.419,0.000,29.707,34.671
1,wh_numeric,0.150,0.533,0.280,0.779,-0.896,1.195
2,gap_numeric,-1.896,0.533,-3.555,0.000,-2.942,-0.851
3,wh_numeric:gap_numeric,-0.052,0.533,-0.097,0.923,-1.097,0.994
4,gap_distance_length,-0.019,0.012,-1.612,0.107,-0.042,0.004
5,wh_numeric:gap_distance_length,-0.002,0.012,-0.161,0.872,-0.025,0.021
6,gap_numeric:gap_distance_length,0.007,0.012,0.576,0.565,-0.016,0.030
7,wh_numeric:gap_numeric:gap_distance_length,0.001,0.012,0.059,0.953,-0.022,0.024



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,26.331,1.595


Gap Postion == PP/Goal

In [14]:
pp_df = sentence_df.copy()
#pp_df['region_surprisal'] = pp_df.apply(lambda x: sum_region_surprisal(x, ['temp_mod']), axis = 1)
pp_df['region_surprisal'] = pp_df.apply(lambda x: sum_region_surprisal(x, ['temp_mod'], 'goal'), axis = 1)

# filter for PP gap
ppgap_df = pp_df[pp_df['gap_position'] == 'IO']

In [15]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(ppgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at pp/goal')


=== GAP DISTANCE: GAP POSITION AT PP/GOAL ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 69.9907
Min. group size: 16
Log-Likelihood: -1135.1754
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,27.083,1.767,15.329,0.000,23.620,30.545
1,gap_distance[T.short],-0.209,1.323,-0.158,0.874,-2.802,2.383
2,gap_distance[T.medium],-0.393,1.323,-0.297,0.766,-2.986,2.200
3,gap_distance[T.long],-0.400,1.323,-0.302,0.762,-2.992,2.193
4,wh_numeric,-0.153,0.935,-0.164,0.870,-1.986,1.680
5,wh_numeric:gap_distance[T.short],0.095,1.323,0.072,0.943,-2.498,2.687
6,wh_numeric:gap_distance[T.medium],0.112,1.323,0.085,0.932,-2.480,2.705
7,wh_numeric:gap_distance[T.long],0.120,1.323,0.091,0.928,-2.473,2.713
8,gap_numeric,3.750,0.935,4.009,0.000,1.917,5.583
9,gap_numeric:gap_distance[T.short],-0.390,1.323,-0.295,0.768,-2.983,2.202



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,44.928,1.975


In [16]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(ppgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at pp/goal')


=== GAP DISTANCE: GAP POSITION AT PP/GOAL ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 68.1078
Min. group size: 16
Log-Likelihood: -1160.9641
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,26.987,1.673,16.131,0.000,23.708,30.266
1,wh_numeric,-0.122,0.734,-0.166,0.868,-1.560,1.316
2,gap_numeric,3.574,0.734,4.871,0.000,2.136,5.012
3,wh_numeric:gap_numeric,-0.038,0.734,-0.052,0.959,-1.476,1.400
4,gap_distance_length,-0.004,0.016,-0.270,0.787,-0.036,0.027
5,wh_numeric:gap_distance_length,0.001,0.016,0.088,0.930,-0.030,0.033
6,gap_numeric:gap_distance_length,-0.003,0.016,-0.162,0.871,-0.034,0.029
7,wh_numeric:gap_numeric:gap_distance_length,0.001,0.016,0.039,0.969,-0.031,0.032



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,45.110,2.003


Embedded Clause

In [17]:
embed = ['subj', 'modifier', 'verb', 'object', 'prep', 'goal', 'temp_mod', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

Embedded Clause - Object Gap Position

In [18]:
# filter for object gap
embed_objectgap_df = sentence_df[sentence_df['gap_position'] == 'DO']

In [19]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(embed_objectgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause object gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE OBJECT GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.2016
Min. group size: 16
Log-Likelihood: -259.2704
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.040,0.175,45.978,0.000,7.697,8.382
1,gap_distance[T.short],0.809,0.071,11.388,0.000,0.669,0.948
2,gap_distance[T.medium],1.011,0.071,14.237,0.000,0.872,1.150
3,gap_distance[T.long],1.859,0.071,26.192,0.000,1.720,1.999
4,wh_numeric,0.021,0.050,0.414,0.679,-0.078,0.119
5,wh_numeric:gap_distance[T.short],-0.052,0.071,-0.739,0.460,-0.192,0.087
6,wh_numeric:gap_distance[T.medium],-0.037,0.071,-0.525,0.600,-0.176,0.102
7,wh_numeric:gap_distance[T.long],-0.036,0.071,-0.504,0.614,-0.175,0.103
8,gap_numeric,-0.382,0.050,-7.602,0.000,-0.480,-0.283
9,gap_numeric:gap_distance[T.short],0.135,0.071,1.903,0.057,-0.004,0.274



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.561,0.428


In [20]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(embed_objectgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause object gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE OBJECT GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.1853
Min. group size: 16
Log-Likelihood: -252.6770
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.125,0.167,48.711,0.000,7.798,8.452
1,wh_numeric,0.001,0.038,0.027,0.978,-0.074,0.076
2,gap_numeric,-0.356,0.038,-9.309,0.000,-0.431,-0.281
3,wh_numeric:gap_numeric,-0.009,0.038,-0.228,0.820,-0.084,0.066
4,gap_distance_length,0.024,0.001,27.817,0.000,0.022,0.025
5,wh_numeric:gap_distance_length,-0.000,0.001,-0.391,0.696,-0.002,0.001
6,gap_numeric:gap_distance_length,0.003,0.001,3.988,0.000,0.002,0.005
7,wh_numeric:gap_numeric:gap_distance_length,0.000,0.001,0.121,0.904,-0.002,0.002



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.527,0.419


Embedded Clause - PP/Goal Gap Position

In [21]:
# filter for PP gap
embed_ppgap_df = sentence_df[sentence_df['gap_position'] == 'IO']

In [22]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(embed_ppgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause pp/goal gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE PP/GOAL GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.2094
Min. group size: 16
Log-Likelihood: -260.7134
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.421,0.144,58.297,0.000,8.138,8.704
1,gap_distance[T.short],0.647,0.072,8.947,0.000,0.506,0.789
2,gap_distance[T.medium],0.818,0.072,11.307,0.000,0.676,0.960
3,gap_distance[T.long],1.625,0.072,22.465,0.000,1.484,1.767
4,wh_numeric,0.128,0.051,2.500,0.012,0.028,0.228
5,wh_numeric:gap_distance[T.short],-0.150,0.072,-2.067,0.039,-0.291,-0.008
6,wh_numeric:gap_distance[T.medium],-0.135,0.072,-1.872,0.061,-0.277,0.006
7,wh_numeric:gap_distance[T.long],-0.134,0.072,-1.852,0.064,-0.276,0.008
8,gap_numeric,-0.087,0.051,-1.698,0.090,-0.187,0.013
9,gap_numeric:gap_distance[T.short],0.056,0.072,0.777,0.437,-0.086,0.198



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.365,0.277


In [23]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(embed_ppgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause pp/goal gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE PP/GOAL GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.1908
Min. group size: 16
Log-Likelihood: -252.9369
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.456,0.136,62.164,0.000,8.190,8.723
1,wh_numeric,0.073,0.039,1.869,0.062,-0.004,0.149
2,gap_numeric,-0.077,0.039,-1.989,0.047,-0.153,-0.001
3,wh_numeric:gap_numeric,0.006,0.039,0.165,0.869,-0.070,0.083
4,gap_distance_length,0.021,0.001,24.233,0.000,0.019,0.023
5,wh_numeric:gap_distance_length,-0.001,0.001,-1.637,0.102,-0.003,0.000
6,gap_numeric:gap_distance_length,0.001,0.001,1.607,0.108,-0.000,0.003
7,wh_numeric:gap_numeric:gap_distance_length,-0.000,0.001,-0.118,0.906,-0.002,0.002



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.340,0.269


#### Double Gaps

In [24]:
sentence_df = pd.read_csv('test_sentences/Double Gaps.csv')

In [25]:
sentence_df[['modifier', 'end']] = sentence_df['modifier'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'licensor', 'subj', 'verb', 'object', 'modifier']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)

Post Gap Region - Modifier

In [26]:
#sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['modifier']), axis = 1)
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['modifier'], 'object'), axis = 1)

In [27]:
iv_list = ['subject_gap', 'object_gap', 'wh_numeric']
mixed_effects_linear_regression(sentence_df, iv_list, 'region_surprisal', 'double gap modifier')


=== DOUBLE GAP MODIFIER ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 22.3603
Min. group size: 8
Log-Likelihood: -484.3765
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,15.003,1.152,13.018,0.000,12.745,17.262
1,subject_gap,2.298,1.057,2.174,0.030,0.226,4.371
2,object_gap,11.564,1.057,10.937,0.000,9.492,13.636
3,subject_gap:object_gap,-2.029,1.495,-1.357,0.175,-4.959,0.902
4,wh_numeric,-0.038,0.748,-0.050,0.960,-1.503,1.428
5,subject_gap:wh_numeric,0.002,1.057,0.002,0.998,-2.070,2.075
6,object_gap:wh_numeric,0.084,1.057,0.079,0.937,-1.989,2.156
7,subject_gap:object_gap:wh_numeric,-0.098,1.495,-0.065,0.948,-3.029,2.833



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,15.385,1.333


Subject Gap Position

In [28]:
subjectgap_df = sentence_df[sentence_df['condition'].isin(['a', 'b', 'd', 'e', 'f', 'h'])]

In [29]:
iv_list = ['wh_numeric', 'gap_numeric']
mixed_effects_linear_regression(subjectgap_df, iv_list, 'region_surprisal', 'double gap modifier: subject gap position')


=== DOUBLE GAP MODIFIER: SUBJECT GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 38.8987
Min. group size: 6
Log-Likelihood: -393.3698
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,18.536,0.846,21.901,0.000,16.878,20.195
1,wh_numeric,-0.040,0.604,-0.067,0.947,-1.224,1.143
2,gap_numeric,3.533,0.604,5.851,0.000,2.350,4.717
3,wh_numeric:gap_numeric,-0.002,0.604,-0.004,0.997,-1.186,1.181



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,7.034,0.769


Object Gap Position

In [30]:
objectgap_df = sentence_df[sentence_df['condition'].isin(['a', 'c', 'd', 'e', 'g', 'h'])]

In [31]:
iv_list = ['wh_numeric', 'gap_numeric']
mixed_effects_linear_regression(objectgap_df, iv_list, 'region_surprisal', 'double gap modifier: object gap position')


=== DOUBLE GAP MODIFIER: OBJECT GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 21.0662
Min. group size: 6
Log-Likelihood: -370.5266
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,20.853,1.191,17.503,0.000,18.518,23.188
1,wh_numeric,-0.020,0.444,-0.045,0.964,-0.891,0.851
2,gap_numeric,5.849,0.444,13.162,0.000,4.978,6.720
3,wh_numeric:gap_numeric,0.018,0.444,0.040,0.968,-0.853,0.889



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,24.438,2.161


Embedded Clause

In [32]:
embed = ['subj', 'object', 'verb', 'modifier', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [33]:
iv_list = ['subject_gap', 'object_gap', 'wh_numeric']
result = mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'double gap embed')
result


=== DOUBLE GAP EMBED ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.1154
Min. group size: 8
Log-Likelihood: -96.5828
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.599,0.140,47.119,0.000,6.325,6.874
1,subject_gap,-0.174,0.076,-2.297,0.022,-0.323,-0.026
2,object_gap,-0.142,0.076,-1.866,0.062,-0.291,0.007
3,subject_gap:object_gap,-0.350,0.107,-3.259,0.001,-0.561,-0.140
4,wh_numeric,-0.010,0.054,-0.195,0.845,-0.116,0.095
5,subject_gap:wh_numeric,0.066,0.076,0.865,0.387,-0.083,0.215
6,object_gap:wh_numeric,-0.008,0.076,-0.107,0.914,-0.157,0.141
7,subject_gap:object_gap:wh_numeric,-0.003,0.107,-0.025,0.980,-0.213,0.208



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.335,0.356


Subject Gap Position

In [34]:
subjectgap_df = sentence_df[sentence_df['condition'].isin(['a', 'b', 'd', 'e', 'f', 'h'])]

In [35]:
iv_list = ['wh_numeric', 'gap_numeric']
mixed_effects_linear_regression(subjectgap_df, iv_list, 'embed_surprisal', 'double gap embedded clause: subject gap position')


=== DOUBLE GAP EMBEDDED CLAUSE: SUBJECT GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.1582
Min. group size: 6
Log-Likelihood: -90.8901
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.389,0.128,50.098,0.000,6.139,6.639
1,wh_numeric,0.020,0.039,0.510,0.610,-0.056,0.095
2,gap_numeric,-0.210,0.039,-5.458,0.000,-0.286,-0.135
3,wh_numeric:gap_numeric,0.030,0.039,0.782,0.434,-0.045,0.106



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.296,0.287


Object Gap Position

In [36]:
objectgap_df = sentence_df[sentence_df['condition'].isin(['a', 'c', 'd', 'e', 'g', 'h'])]

In [37]:
iv_list = ['wh_numeric', 'gap_numeric']
mixed_effects_linear_regression(objectgap_df, iv_list, 'embed_surprisal', 'double gap embedded clause: object gap position')


=== DOUBLE GAP EMBEDDED CLAUSE: OBJECT GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.1609
Min. group size: 6
Log-Likelihood: -93.8563
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.397,0.143,44.832,0.000,6.118,6.677
1,wh_numeric,0.001,0.039,0.031,0.976,-0.075,0.077
2,gap_numeric,-0.202,0.039,-5.201,0.000,-0.278,-0.126
3,wh_numeric:gap_numeric,0.012,0.039,0.300,0.764,-0.064,0.088



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.377,0.357


#### Wh-Islands

In [38]:
sentence_df = pd.read_csv('test_sentences/Wh-Islands.csv')
sentence_df['comp_type'] = sentence_df['comp_type'].fillna('null')

In [39]:
sentence_df[['cont', 'end']] = sentence_df['cont'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'filler_word', 'subj1', 'verb1', 'comp', 'subj2', 'embed_verb', 'obj', 'cont']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence (x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['comp_type'] = pd.Categorical(sentence_df['comp_type'], categories = ['null', 'that', 'whether'], ordered = True)

Post-Gap Region - Continuation

In [40]:
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['cont'], 'obj'), axis = 1)

In [41]:
iv_list = ['wh_numeric', 'gap_numeric', 'comp_type'] # island_type = comp_type
mixed_effects_linear_regression(sentence_df, iv_list, 'region_surprisal', 'wh-islands continuation')


=== WH-ISLANDS CONTINUATION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 38.1849
Min. group size: 12
Log-Likelihood: -784.1198
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,26.430,1.226,21.561,0.000,24.028,28.833
1,comp_type[T.that],-0.107,0.977,-0.109,0.913,-2.022,1.808
2,comp_type[T.whether],-0.150,0.977,-0.153,0.878,-2.065,1.765
3,wh_numeric,0.002,0.691,0.002,0.998,-1.353,1.356
4,wh_numeric:comp_type[T.that],-0.058,0.977,-0.059,0.953,-1.973,1.857
5,wh_numeric:comp_type[T.whether],-0.090,0.977,-0.092,0.927,-2.005,1.825
6,gap_numeric,4.975,0.691,7.201,0.000,3.621,6.329
7,gap_numeric:comp_type[T.that],0.119,0.977,0.122,0.903,-1.796,2.034
8,gap_numeric:comp_type[T.whether],-0.053,0.977,-0.054,0.957,-1.968,1.862
9,wh_numeric:gap_numeric,-0.100,0.691,-0.144,0.885,-1.454,1.254



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,20.508,1.299


Embedded Clause

In [42]:
embed = ['subj1', 'verb1', 'comp', 'subj2', 'embed_verb', 'obj', 'cont', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [43]:
iv_list = ['wh_numeric', 'gap_numeric', 'comp_type'] # island_type = comp_type
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'wh-islands embed')


=== WH-ISLANDS EMBED ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 0.0370
Min. group size: 12
Log-Likelihood: -27.8372
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.800,0.213,41.291,0.000,8.382,9.218
1,comp_type[T.that],-0.058,0.030,-1.896,0.058,-0.117,0.002
2,comp_type[T.whether],0.072,0.030,2.353,0.019,0.012,0.131
3,wh_numeric,-0.169,0.021,-7.846,0.000,-0.211,-0.127
4,wh_numeric:comp_type[T.that],0.002,0.030,0.074,0.941,-0.057,0.062
5,wh_numeric:comp_type[T.whether],-0.024,0.030,-0.791,0.429,-0.084,0.036
6,gap_numeric,-0.006,0.021,-0.302,0.763,-0.049,0.036
7,gap_numeric:comp_type[T.that],-0.012,0.030,-0.379,0.705,-0.071,0.048
8,gap_numeric:comp_type[T.whether],-0.013,0.030,-0.415,0.678,-0.072,0.047
9,wh_numeric:gap_numeric,-0.033,0.021,-1.553,0.120,-0.076,0.009



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.899,1.589


#### Adjunct Islands 

In [44]:
sentence_df = pd.read_csv('test_sentences/Adjunct Islands.csv')
sentence_df['adjunct_type'] = sentence_df['adjunct_type'].fillna('null')

In [45]:
sentence_df[['cont', 'end']] = sentence_df['cont'].apply(split_ends)
sentence_df['adjunct_front_trigger'] = np.nan
sentence_df['adjunct_front_trigger'] = sentence_df['adjunct_trigger'].where(sentence_df['adjunct_type'] == 'front', sentence_df['adjunct_front_trigger'])
sentence_df['adjunct_trigger'] = sentence_df['adjunct_trigger'].where(sentence_df['adjunct_type'] != 'front', np.nan)
sentence_df.rename(columns = {'adjunct_trigger': 'adjunct_back_trigger'}, inplace = True)
sentence_df = sentence_df[[col for col in sentence_df.columns if col != 'end'] + ['end']]

In [46]:
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'filler_word', 'adjunct_back_trigger', 'prep', 'subj', 'mod', 'verb', 'obj', 'cont', 'adjunct_front_trigger']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['adjunct_type'] = pd.Categorical(sentence_df['adjunct_type'], categories = ['null', 'back', 'front'], ordered = True)

Post-Gap Region - Continuation

In [47]:
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['cont'], 'obj'), axis = 1)

In [48]:
iv_list = ['wh_numeric', 'gap_numeric', 'adjunct_type'] # island_type = adjunct_type
mixed_effects_linear_regression(sentence_df, iv_list, 'region_surprisal', 'adjunct islands continuation')


=== ADJUNCT ISLANDS CONTINUATION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 41.1138
Min. group size: 12
Log-Likelihood: -799.5044
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,37.730,1.731,21.798,0.000,34.338,41.123
1,adjunct_type[T.back],-0.050,1.014,-0.050,0.960,-2.038,1.937
2,adjunct_type[T.front],0.087,1.014,0.085,0.932,-1.900,2.074
3,wh_numeric,0.046,0.717,0.064,0.949,-1.359,1.451
4,wh_numeric:adjunct_type[T.back],-0.030,1.014,-0.029,0.977,-2.017,1.957
5,wh_numeric:adjunct_type[T.front],0.006,1.014,0.006,0.995,-1.981,1.993
6,gap_numeric,5.065,0.717,7.066,0.000,3.660,6.471
7,gap_numeric:adjunct_type[T.back],0.008,1.014,0.008,0.993,-1.979,1.995
8,gap_numeric:adjunct_type[T.front],-0.024,1.014,-0.024,0.981,-2.011,1.963
9,wh_numeric:gap_numeric,-0.036,0.717,-0.050,0.960,-1.441,1.369



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,49.640,2.804


Embedded Clause

In [49]:
embed = ['adjunct_back_trigger', 'prep', 'subj', 'mod', 'verb', 'obj', 'cont', 'adjunct_front_trigger', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [50]:
iv_list = ['wh_numeric', 'gap_numeric', 'adjunct_type'] # island_type = adjunct_type
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'adjunct islands embed')


=== ADJUNCT ISLANDS EMBED ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 0.1205
Min. group size: 12
Log-Likelihood: -152.7436
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.656,0.231,37.484,0.000,8.203,9.108
1,adjunct_type[T.back],0.740,0.055,13.472,0.000,0.632,0.847
2,adjunct_type[T.front],0.813,0.055,14.803,0.000,0.705,0.920
3,wh_numeric,0.013,0.039,0.323,0.747,-0.064,0.089
4,wh_numeric:adjunct_type[T.back],0.019,0.055,0.341,0.733,-0.089,0.126
5,wh_numeric:adjunct_type[T.front],0.095,0.055,1.739,0.082,-0.012,0.203
6,gap_numeric,-0.311,0.039,-8.010,0.000,-0.387,-0.235
7,gap_numeric:adjunct_type[T.back],0.127,0.055,2.309,0.021,0.019,0.234
8,gap_numeric:adjunct_type[T.front],0.134,0.055,2.440,0.015,0.026,0.242
9,wh_numeric:gap_numeric,-0.002,0.039,-0.058,0.954,-0.078,0.074



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.036,1.021


#### Complex NP Islands

In [ ]:
sentence_df = pd.read_csv('test_sentences/Complex NP Islands.csv')

In [ ]:
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
sentence_df['sentence'] = sentence_df.apply(merge_sentence, axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df

Subject Condition

In [ ]:
subject_df = sentence_df.copy()

In [ ]:
subject_df['region_surprisal'] = subject_df.apply(lambda x: sum_region_surprisal(x, ['subj_setup']), axis = 1)
subject_df

In [ ]:
# filter for subject condition
subject_df = subject_df[subject_df['subj_obj'] == 'subject']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(subject_df, iv_list, 'region_surprisal', 'complex np islands: subject condition')

Object Condition

In [ ]:
object_df = sentence_df.copy()

In [ ]:
object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['end']), axis = 1)
object_df

In [ ]:
# filter for object gap
object_df = object_df[object_df['subj_obj'] == 'object']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(object_df, iv_list, 'region_surprisal', 'complex np islands: object condition')

Embedded Clause

In [ ]:
embed = ['subj', 'that_rc', 'what_rc', 'rc_np', 'prep', 'prep_np', 'subj_setup', 'obj_setup', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)
sentence_df

Embedded Clause - Subject Condition

In [ ]:
# filter for subject condition
subject_df = sentence_df[sentence_df['subj_obj'] == 'subject']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(subject_df, iv_list, 'embed_surprisal', 'complex np islands: embedded clause subject condition')

Embedded Clause - Object Condition

In [ ]:
# filter for object gap
object_df = sentence_df[sentence_df['subj_obj'] == 'object']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(object_df, iv_list, 'embed_surprisal', 'complex np islands: embedded clause object condition')